<a href="https://colab.research.google.com/github/AshvinVignesh/Incident-Root-Cause-Analysis-Deep-Learning-Classifcation/blob/main/Incident_Root_Cause_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Incident Root Cause Analysis

In [3]:
import pandas as pd
import numpy as np 
import os 
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split

In [4]:
data = pd.read_csv("root_cause_analysis.csv")
data.head()

,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


In [6]:
label_encoder = preprocessing.LabelEncoder()
data['ROOT_CAUSE'] = label_encoder.fit_transform(data['ROOT_CAUSE'])

In [8]:
np_data = data.to_numpy()
np_data

array([[   1,    0,    0, ...,    0,    1,    1],
       [   2,    0,    0, ...,    0,    1,    1],
       [   3,    0,    1, ...,    1,    1,    1],
       ...,
       [ 998,    1,    1, ...,    0,    0,    1],
       [ 999,    0,    1, ...,    0,    0,    2],
       [1000,    1,    0, ...,    1,    0,    0]])

In [9]:
np_data = data.to_numpy().astype(float)
np_data

array([[   1.,    0.,    0., ...,    0.,    1.,    1.],
       [   2.,    0.,    0., ...,    0.,    1.,    1.],
       [   3.,    0.,    1., ...,    1.,    1.,    1.],
       ...,
       [ 998.,    1.,    1., ...,    0.,    0.,    1.],
       [ 999.,    0.,    1., ...,    0.,    0.,    2.],
       [1000.,    1.,    0., ...,    1.,    0.,    0.]])

In [12]:
X_data = np_data[:,1:8]
Y_data = np_data[:,8]


## one-hot encoding

In [13]:
import tensorflow as tf


In [14]:
Y_data = tf.keras.utils.to_categorical(Y_data,3)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X_data,Y_data,test_size=0.2)

In [17]:
print(label_encoder.classes_)

['DATABASE_ISSUE' 'MEMORY_LEAK' 'NETWORK_DELAY']


In [23]:
from tensorflow import keras
verbose = 1
batch = 64
epoch = 20
hidden_nodes = 128
output_node = len(label_encoder.classes_)
validation_split = 0.2


model= tf.keras.models.Sequential()

model.add(keras.layers.Dense(  hidden_nodes,  input_shape=(7,),  name ="HIDDEN-LAYER-1"  , activation ="relu" ))

model.add(keras.layers.Dense(  hidden_nodes,  name= "HIDDEN-LAYER-2",  activation= "relu" ))

model.add(keras.layers.Dense(  output_node,  name= "OUTPUT-LAYER" , activation = "softmax"))

model.compile(loss= "categorical_crossentropy" , metrics = ['accuracy'])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HIDDEN-LAYER-1 (Dense)      (None, 128)               1024      
                                                                 
 HIDDEN-LAYER-2 (Dense)      (None, 128)               16512     
                                                                 
 OUTPUT-LAYER (Dense)        (None, 3)                 387       
                                                                 
Total params: 17,923
Trainable params: 17,923
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train,Y_train, verbose=verbose , batch_size= batch, epochs=epoch , validation_split = validation_split)

Epoch 1/20
10/10 [==============================] - 1s 25ms/step - loss: 0.9232 - accuracy: 0.6594 - val_loss: 0.7705 - val_accuracy: 0.8125
Epoch 2/20
10/10 [==============================] - 0s 4ms/step - loss: 0.6935 - accuracy: 0.7984 - val_loss: 0.6363 - val_accuracy: 0.7750
Epoch 3/20
10/10 [==============================] - 0s 4ms/step - loss: 0.5694 - accuracy: 0.8031 - val_loss: 0.5621 - val_accuracy: 0.8062
Epoch 4/20
10/10 [==============================] - 0s 6ms/step - loss: 0.5150 - accuracy: 0.8109 - val_loss: 0.5188 - val_accuracy: 0.7937
Epoch 5/20
10/10 [==============================] - 0s 6ms/step - loss: 0.4937 - accuracy: 0.8109 - val_loss: 0.5154 - val_accuracy: 0.7937
Epoch 6/20
10/10 [==============================] - 0s 4ms/step - loss: 0.4830 - accuracy: 0.8188 - val_loss: 0.5126 - val_accuracy: 0.7937
Epoch 7/20
10/10 [==============================] - 0s 4ms/step - loss: 0.4692 - accuracy: 0.8313 - val_loss: 0.5211 - val_accuracy: 0.8000
Epoch 8/20
10/10 [=

In [25]:
print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)


Evaluation against Test Dataset :
------------------------------------
7/7 [==============================] - 0s 2ms/step - loss: 0.3204 - accuracy: 0.8750


[0.3203834593296051, 0.875]

## Predicting Root Causes

In [26]:
print(label_encoder.inverse_transform(np.argmax(
        model.predict([[1,0,0,0,1,1,0],
                       [0,1,1,1,0,0,0],
                       [1,1,0,1,1,0,1],
                       [0,0,0,0,0,1,0],
                       [1,0,1,0,1,1,1]]), axis=1 )))

1/1 [==============================] - 0s 79ms/step
['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY_LEAK' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']
